<a href="https://colab.research.google.com/github/Divyaanshvats/finrir/blob/main/FENRIR_SECURITY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import time
import json
from tqdm import tqdm

In [2]:
TAGS = ["bash", "git", "tar", "grep", "venv"]
BASE_URL = "https://api.stackexchange.com/2.3"
SITE = "stackoverflow"

In [3]:
import requests
import time
import json
from tqdm import tqdm

def get_questions(tag, pagesize=50):
    url = f"{BASE_URL}/questions"
    params = {
        "order": "desc",
        "sort": "votes",
        "tagged": tag,
        "site": SITE,
        "pagesize": pagesize,
        "filter": "!9_bDDxJY5",  # includes body
    }
    response = requests.get(url, params=params)
    try:
        return response.json().get("items", [])
    except json.JSONDecodeError as e:
        print("Error decoding JSON:")
        print(response.text)
        raise e

In [4]:
def get_best_answer(question_id):
    url = f"{BASE_URL}/questions/{question_id}/answers"
    params = {
        "order": "desc",
        "sort": "votes",
        "site": SITE,
        "filter": "!9_bDE(fI5",  # includes body
    }
    response = requests.get(url, params=params)
    if not response.text:  # Check for empty response
        print(f"Empty response received for question ID {question_id}. Possible rate limit issue.")
        return None
    answers = response.json().get("items", [])
    if not answers:
        return None
    return answers[0]["body"]

In [5]:
from bs4 import BeautifulSoup

data = []

for tag in TAGS:
    print(f"Collecting questions for tag: {tag}")
    questions = get_questions(tag, pagesize=60)
    for q in tqdm(questions):
        if not q.get("is_answered"):
            continue
        q_text = BeautifulSoup(q["title"], "html.parser").get_text()
        a_html = get_best_answer(q["question_id"])
        if not a_html:
            continue
        a_text = BeautifulSoup(a_html, "html.parser").get_text()
        data.append({
            "question": q_text.strip(),
            "answer": a_text.strip()
        })
        time.sleep(1)  # increased sleep time

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [6]:
import os

os.makedirs("data", exist_ok=True)

with open("data/questions.json", "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

print(f"✅ Collected {len(data)} Q&A pairs.")

✅ Collected 0 Q&A pairs.


In [7]:
# Install required packages
!pip install transformers peft accelerate datasets evaluate bitsandbytes --quiet

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, TaskType

# Load tokenizer and base model
base_model_id = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(base_model_id)

# ❗ Load base model ONLY
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto",
    torch_dtype="auto"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
from datasets import load_dataset
import json

# Replace with your dataset path
dataset_path = "/content/data/questions.json"

# Load the data from the JSON file
with open(dataset_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# Create a dataset from the loaded data
from datasets import Dataset
dataset = Dataset.from_list(data)

# Format for fine-tuning: instruction tuning style
def format_example(example):
    return {
        "text": f"### Instruction:\n{example['question']}\n\n### Response:\n{example['answer']}"
    }

dataset = dataset.map(format_example)

In [10]:
from peft import LoraConfig, TaskType, get_peft_model

In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, TaskType, get_peft_model

base_model_id = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(base_model_id)

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto",
    torch_dtype="auto"
)

# Apply LoRA
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj", "v_proj"]
)

# Wrap the base model with LoRA
model = get_peft_model(base_model, peft_config)

# Confirm
print(type(model))  # ✅ should show PeftModelForCausalLM

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

<class 'peft.peft_model.PeftModelForCausalLM'>


In [12]:
from transformers import DataCollatorForLanguageModeling

# Set padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)

# ✅ Keep the "text" field — we use it during training
tokenized_dataset = dataset.map(tokenize, batched=True)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [13]:
from transformers import TrainingArguments, Trainer

model.print_trainable_parameters()  # Should show only LoRA params are trainable

training_args = TrainingArguments(
    output_dir="./phi2-lora",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    logging_steps=10,
    save_strategy="epoch",
    fp16=True,
    report_to="none"
)

trainable params: 2,621,440 || all params: 2,782,305,280 || trainable%: 0.0942


In [14]:
trainer = Trainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=training_args,
    data_collator=data_collator
)

trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


ValueError: No columns in the dataset match the model's forward method signature: ({', '.join(signature_columns)}). The following columns have been ignored: []. Please check the dataset and model. You may need to set `remove_unused_columns=False` in `TrainingArguments`.

In [15]:
model.save_pretrained("phi2-lora-adapter")
tokenizer.save_pretrained("phi2-lora-adapter")

('phi2-lora-adapter/tokenizer_config.json',
 'phi2-lora-adapter/special_tokens_map.json',
 'phi2-lora-adapter/vocab.json',
 'phi2-lora-adapter/merges.txt',
 'phi2-lora-adapter/added_tokens.json',
 'phi2-lora-adapter/tokenizer.json')

In [16]:
!ls phi2-lora-adapter

adapter_config.json	   merges.txt		    tokenizer_config.json
adapter_model.safetensors  README.md		    tokenizer.json
added_tokens.json	   special_tokens_map.json  vocab.json


In [17]:
import os

# Save the PEFT model, which includes the adapter weights and adapter_config.json
model.save_pretrained("phi2-lora-adapter")
# Save the tokenizer
tokenizer.save_pretrained("phi2-lora-adapter")

# Remove the unnecessary line saving the base model config
# model.config.save_pretrained("phi2-lora-adapter")

# List the contents of the directory to confirm files are saved
print("Contents of phi2-lora-adapter after saving:")
print(os.listdir("phi2-lora-adapter"))

Contents of phi2-lora-adapter after saving:
['added_tokens.json', 'tokenizer_config.json', 'merges.txt', 'special_tokens_map.json', 'tokenizer.json', 'adapter_config.json', 'README.md', 'adapter_model.safetensors', 'vocab.json']


In [18]:
!ls -lh phi2-lora-adapter

total 15M
-rw-r--r-- 1 root root  769 Jun 18 13:49 adapter_config.json
-rw-r--r-- 1 root root  11M Jun 18 13:49 adapter_model.safetensors
-rw-r--r-- 1 root root 1.1K Jun 18 13:49 added_tokens.json
-rw-r--r-- 1 root root 446K Jun 18 13:49 merges.txt
-rw-r--r-- 1 root root 5.0K Jun 18 13:49 README.md
-rw-r--r-- 1 root root  473 Jun 18 13:49 special_tokens_map.json
-rw-r--r-- 1 root root 7.3K Jun 18 13:49 tokenizer_config.json
-rw-r--r-- 1 root root 3.4M Jun 18 13:49 tokenizer.json
-rw-r--r-- 1 root root 780K Jun 18 13:49 vocab.json


In [19]:
!ls phi2-lora-adapter

adapter_config.json	   merges.txt		    tokenizer_config.json
adapter_model.safetensors  README.md		    tokenizer.json
added_tokens.json	   special_tokens_map.json  vocab.json


In [20]:
!python agent.py "Create a new Git branch and switch to it"

python3: can't open file '/content/agent.py': [Errno 2] No such file or directory


In [21]:
!tail -n 1 logs/trace.jsonl

tail: cannot open 'logs/trace.jsonl' for reading: No such file or directory


In [ ]:
import sys
import os
import json
import datetime
import subprocess
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# === CONFIG ===
base_model = "microsoft/phi-2"
adapter_path = "./phi2-lora-adapter"

tokenizer = AutoTokenizer.from_pretrained(adapter_path)
base = AutoModelForCausalLM.from_pretrained(base_model, device_map="auto", torch_dtype=torch.float16)
model = PeftModel.from_pretrained(base, adapter_path, is_trainable=False)

# === Get prompt ===
prompt = sys.argv[1] if len(sys.argv) > 1 else input("Enter instruction:\n")

# === Format for inference ===
formatted = f"### Instruction:\n{prompt}\n\n### Response:\n"
inputs = tokenizer(formatted, return_tensors="pt").to(model.device)

# === Generate output ===
with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=100)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# === Extract response only
if "### Response:" in response:
    response = response.split("### Response:")[-1].strip()

print("\n📋 Plan Generated:\n" + response)

# === Dry-run first line if shell command
first_line = response.strip().split('\n')[0]

def is_shell_command(text):
    return any(text.startswith(cmd) for cmd in ["git ", "tar ", "grep ", "find ", "python ", "./", "ls ", "cd ", "mkdir ", "echo ", "rm ", "curl ", "wget ", "pip ", "bash ", "source "])

if is_shell_command(first_line):
    print("\n🧪 Dry-run:")
    subprocess.run(f"echo {first_line}", shell=True)

# === Save logs
log = {
    "timestamp": datetime.datetime.now().isoformat(),
    "prompt": prompt,
    "response": response,
    "dry_run_command": first_line if is_shell_command(first_line) else None
}

os.makedirs("logs", exist_ok=True)
with open("logs/trace.jsonl", "a", encoding="utf-8") as f:
    f.write(json.dumps(log) + "\n")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
!python agent.py "Create a new Git branch and switch to it"

In [ ]:
!tail -n 1 logs/trace.jsonl

In [ ]:
!python agent.py "Create a new Git branch and switch to it"

In [ ]:
!python agent.py "Compress the folder reports into reports.tar.gz"

In [ ]:
!python agent.py "List all Python files in the current directory recursively"

In [ ]:
!python agent.py "Set up a virtual environment and install requests"

In [ ]:
!python agent.py "Fetch only the first ten lines of a file named output.log"

In [ ]:
!python agent.py "List all Python files in the current directory recursively"

In [ ]:
!python agent.py "Set up a virtual environment and install requests"

In [ ]:
!python agent.py "Find lines containing 'error' in all .log files, case-insensitive"

In [ ]:
!python agent.py "Delete all .tmp files in a directory with confirmation"